In [ ]:
import numpy as np

In [ ]:
def read_text(path):
  with open(path) as f:
    text = f.read()
  return text

In [ ]:
def sep_punc(text):
  toks = []
  for token in nlp(text):
    if token.text not in '\n\n\n!"#$%&()*+,-./:;<  --=>?@[\\]^_`{|}~\t\n':
      toks.append(token.text.lower())   
  return toks   

In [ ]:
import spacy 
nlp = spacy.load('en_core_web_sm')

In [ ]:
# path = "/content/moby_dick_four_chapters.txt"
path = '/content/hp p1.txt'
file_text = read_text(path)
tokens = sep_punc(file_text)
# print(tokens )

In [ ]:
train_len = 40 + 1
text_sequences = []
for i in range(train_len , len(tokens)):
  seq = tokens[i - train_len : i]
  text_sequences.append(seq)


In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(filters = '\n\n\n!"#$%&()*+,-./:;<  --=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)
# print(sequences)

In [ ]:
ind_word = tokenizer.index_word

In [ ]:
vocab_size = len(tokenizer.word_counts)
vocab_size

5860

In [ ]:
from keras.utils import to_categorical
sequences = np.array(sequences)
X = sequences[: , :-1]
y = sequences[: , -1]
y = to_categorical(y , num_classes = vocab_size + 1) # + 1 for holding zeroth class

seq_len = X.shape[1]
X.shape

(81197, 40)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Embedding , LSTM , Dropout

In [ ]:
def create_model(vocab_size , seq_len):
  model = Sequential()
  model.add(Embedding(input_dim = vocab_size , output_dim= seq_len * 2, input_length = seq_len))
  model.add(LSTM(units= seq_len * 2 , return_sequences= True))
  model.add(LSTM(units = seq_len*2))
  model.add(Dense(seq_len*3 , activation = 'relu'))
  model.add(Dropout(0.2))
  model.add(Dense(vocab_size , activation = 'softmax'))
  model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])
  model.summary()
  return model

In [ ]:
model = create_model(vocab_size + 1 , seq_len)
model.fit(X, y , batch_size = 256 , epochs = 100)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 80)            468880    
_________________________________________________________________
lstm_4 (LSTM)                (None, 40, 80)            51520     
_________________________________________________________________
lstm_5 (LSTM)                (None, 80)                51520     
_________________________________________________________________
dense_4 (Dense)              (None, 120)               9720      
_________________________________________________________________
dropout_2 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5861)              709181    
Total params: 1,290,821
Trainable params: 1,290,821
Non-trainable params: 0
____________________________________________

In [ ]:
from keras.preprocessing.sequence import pad_sequences
def generate_text(model , tokenizer , seq_len , seed_text, no_of_words):
  output_text = []
  input_text = seed_text

  for i in range(no_of_words):
    encoded_text = tokenizer.texts_to_sequences([input_text])[0]
    pad_encoded = pad_sequences([encoded_text] , maxlen = seq_len , truncating = 'pre')

    ind = np.argmax(model.predict(pad_encoded) , axis = 1)[0]
    # type(ind)
    word = tokenizer.index_word[ind]
    input_text += ' ' + word

    output_text.append(word)

  return ' '.join(output_text)

In [ ]:
import random
r_p = random.randint(0 , len(text_sequences))
rand_text = text_sequences[r_p]
seed_text = ' '.join(rand_text)

# seed_text
new = generate_text(model , tokenizer ,seq_len , seed_text , 40)
# print(seed_text + "\n" + new )
seed_text

"the zoo director himself made aunt petunia a cup of strong sweet tea while he apologized over and over again piers and dudley could only gibber as far as harry had seen the snake had n't done anything except snap playfully"

In [ ]:
# seed_text = 'harry did not wanted to join the slytherine he knew about people who were particularly bad and belonged to slytherine'
generate_text(model , tokenizer , seq_len , seed_text , 40)

"at their heels as it was a very good mood in the door i 'm not going to be fair to be able to breathe though he was n't sure he was n't sure he was n't sure he was"